**Group No. 10**

Aarti Anil Zikre

Andrews Truman

Premkumar Janakbhai Patel

Vitthlesh Sheth

**Objective:** To understand and gain insights from a retail dataset by performing various exploratory data analyses, data visualization, and data modelling.

**Dataset Columns:**

- **InvoiceNo:** Invoice number. A unique number per invoice.
- **StockCode:** Product code. A unique number per product.
- **Description:** Product description.
- **Quantity:** The number of products sold per invoice.
- **InvoiceDate:** The date and time of the invoice.
- **UnitPrice:** The price of one unit of the product.
- **CustomerID:** Customer identification number.
- **Country:** The country where the customer resides.


## 1. Data Preprocessing and Cleaning:


1.1. Import necessary libraries and read the dataset:


To get started, we'll first need to import the required libraries and load the dataset we plan to work with. The 'InvoiceDate' attribute is currently in a format we don't want, so we'll need to change it to our preferred format. Additionally, we've noticed some spelling errors and extra spaces in our text data, which we want to clean up. For the next steps, we'd like all our text data to be in uppercase.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/DAB303/Project2/Sales_data.csv', encoding='unicode_escape', dtype= {'CustomerID': 'Int64'})

1.2. Display the top 10 rows of the dataframe:

In [ ]:
df.head(10)

1.3. Check for missing values:

In [ ]:
missing_values = df.isnull().sum()
print(missing_values)

# Drop missing value
Data = df.dropna()

In [ ]:
#Formatting Date/Time
Data['InvoiceDate'] = pd.to_datetime(Data['InvoiceDate'], format = '%m/%d/%Y %H:%M')

#Strings
Data['Description'] = Data['Description'].str.replace('.','').str.upper().str.strip()
Data['Description'] = Data['Description'].replace('\s+',' ',regex = True)
Data['InvoiceNo'] = Data['InvoiceNo'].astype(str).str.upper()
Data['StockCode'] = Data['StockCode'].str.upper()
Data['Country'] = Data['Country'].str.upper()
Data.head()

In [ ]:
#Listing Some Irrelevant StockCodes
Irrelevant = Data['StockCode'].unique()
Irrelevant.sort()
print('Irrelevant Transactions: \n',Irrelevant[::-1][:4])
#Quantity and UnitPrice Summary
Data.describe().iloc[:,:2]


In the dataset summary, it's clear that we have some unusual and irregular values in the 'UnitPrice' and 'Quantity' columns. To ensure they don't skew our analysis, we'll identify and remove these outliers.

Furthermore, in the 'StockCode' variable, we've noticed that some transactions are not related to products but instead represent costs or fees associated with postage, banking, or other non-essential transactions. These non-product transactions are unnecessary for our analysis, and we'll exclude them from our dataset.

In [ ]:
import scipy as sp
import numpy as np

# Outliers and Irrelevant Values
# Dropping all stockcodes that contain only strings
Data = Data[~Data['StockCode'].str.isalpha()]
Data.reset_index(drop=True, inplace=True)

# Removing Outliers Based on Z-score
Data = Data[(np.abs(sp.stats.zscore(Data['UnitPrice'])) < 3) & (np.abs(sp.stats.zscore(Data['Quantity'])) < 5)]



We've observed that some transactions involve returned products, indicated by a 'c' character at the beginning of the 'InvoiceNo' and a negative 'UnitPrice.' However, our data contains errors where purchases have negative 'UnitPrice' and vice versa. We need to correct these discrepancies and also address cases where the 'UnitPrice' is not specified.

In the 'Description' attribute, there are numerous missing or incorrect values. To rectify this issue, we will:

Remove transactions with no available description.
For other missing descriptions, we'll cross-reference the 'Description' based on the product's 'StockCode' and fill in the missing values with the correct 'Description' obtained from other transactions with the same 'StockCode'. This way, we'll have more complete and accurate descriptions for our products.

In [ ]:
# Missing & Incorrect Values
Data.drop(Data[(Data.Quantity>0) & (Data.InvoiceNo.str.contains('C') == True)].index, inplace = True)
Data.drop(Data[(Data.Quantity<0) & (Data.InvoiceNo.str.contains('C') == False)].index, inplace = True)
Data.drop(Data[Data.Description.str.contains('?',regex=False) == True].index, inplace = True)
Data.drop(Data[Data.UnitPrice == 0].index, inplace = True)

for index,value in Data.StockCode[Data.Description.isna()==True].items():
    if pd.notna(Data.Description[Data.StockCode == value]).sum() != 0:
        Data.Description[index] = Data.Description[Data.StockCode == value].mode()[0]
    else:
        Data.drop(index = index, inplace = True)

Data['Description'] = Data['Description'].astype(str)

In our dataset, we've noticed that some rows have different 'UnitPrices' for the same products. This variation could be due to discounts, special customer conditions, or even input errors. There are instances where the same customer is charged different prices on different days, or different customers are charged different prices for the same product on the same day.

For the purpose of this analysis, we'll assume that these inconsistent values are incorrect, likely resulting from input errors or human mistakes. To standardize the 'UnitPrice' for each product, we'll use the mode (most common) value as the reference price. This way, we'll have a more consistent and reliable 'UnitPrice' for our analysis.

In [ ]:
#Incorrect Prices
StockList = Data.StockCode.unique()
CalculatedMode = map(lambda x: Data.UnitPrice[Data.StockCode == x].mode()[0],StockList)
StockModes = list(CalculatedMode)
for i,v in enumerate(StockList):
    Data.loc[Data['StockCode']== v, 'UnitPrice'] = StockModes[i]

We will add two new features: 'Final Price' calculated from 'UnitPrice' and 'Quantity,' and extract 'Month' and 'Day of the Week' from 'InvoiceDate' for time-based analysis.

We'll also correct incorrect customer IDs by grouping data by 'CustomerID' and replacing any IDs associated with multiple countries with the mode (most common) country for that customer.

In [ ]:
#Customers with Different Countries
Customers = Data.groupby('CustomerID')['Country'].unique()
Customers.loc[Customers.apply(lambda x:len(x)>1)]

In [ ]:
#Fixing Duplicate CustomerIDs
for i,v in Data.groupby('CustomerID')['Country'].unique().items():
    if len(v)>1:
        Data.Country[Data['CustomerID'] == i] = Data.Country[Data['CustomerID'] == i].mode()[0]

#Adding Desired Features
Data['FinalPrice'] = Data['Quantity']*Data['UnitPrice']
Data['InvoiceMonth'] = Data['InvoiceDate'].apply(lambda x: x.strftime('%B'))
Data['Day of week'] = Data['InvoiceDate'].dt.day_name()

#Exporting Processed Data
Data.to_csv('OnlineRetail_Cleaned.csv', date_format = '%Y-%m-%d %H:%M', index=False)

## 2. Exploratory Data Analysis:


In this section, we'll visualize the data and create informative reports and dashboards to gain a better understanding of our dataset. Here's how we'll approach it:

Import the data and set the index to datetime for time series analysis.
First, we'll identify and display the top products that were sold globally. We'll use two key indicators to measure their significance:


2.1 In the first plot, we'll showcase the top 20 products that were purchased in the highest quantities by customers.
In the second plot, we'll highlight the products that have generated the most revenue for us.
These visualizations will help us gain insights into our top-performing products both in terms of sales volume and monetary benefits.

In [ ]:
#importing necessary libraries and the cleaned dataset
import matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline

Data_Cleaned = pd.read_csv('/content/drive/MyDrive/DAB303/Project2/OnlineRetail_Cleaned.csv', index_col = 'InvoiceDate')
Data_Cleaned.index = pd.to_datetime(Data_Cleaned.index, format = '%Y-%m-%d %H:%M')


#top 20 products by quantity and finalprice
sns.set_style('whitegrid')
Top20Quan = Data_Cleaned.groupby('Description')['Quantity'].agg('sum').sort_values(ascending=False)[0:20]
Top20Price = Data_Cleaned.groupby('Description')['FinalPrice'].agg('sum').sort_values(ascending=False)[0:20]
#creating the subplot
fig,axs = plt.subplots(nrows=2, ncols=1, figsize = (12,12))
plt.subplots_adjust(hspace = 0.3)
fig.suptitle('Best Selling Products by Amount and Value', fontsize=15, x = 0.4, y = 0.98)
sns.barplot(x=Top20Quan.values, y=Top20Quan.index, ax= axs[0]).set(xlabel='Total amount of sales')
axs[0].set_title('By Amount', size=12, fontweight = 'bold')
sns.barplot(x=Top20Price.values, y=Top20Price.index, ax= axs[1]).set(xlabel='Total value of sales')
axs[1].set_title('By Value', size=12, fontweight = 'bold')
plt.show()

2.2 In this section, our focus is on understanding product returns and customer behavior related to returns. We'll first identify which products are returned most frequently by our customers, providing insights into less-satisfactory items. Additionally, we'll analyze which customers and countries have the highest number of returned items, helping us uncover trends in customer return behavior and geographic variations in return patterns.

In [ ]:
#finding the most returned items and the customers with the corresponding country
ReturnedItems = Data_Cleaned[Data_Cleaned.Quantity<0].groupby('Description')['Quantity'].sum()
ReturnedItems = ReturnedItems.abs().sort_values(ascending=False)[0:10]
ReturnCust = Data_Cleaned[Data_Cleaned.Quantity<0].groupby(['CustomerID','Country'])['Quantity'].sum()
ReturnCust = ReturnCust.abs().sort_values(ascending=False)[0:10]
#creting the subplot
fig, [ax1, ax2] = plt.subplots(nrows=2, ncols=1, figsize=(12,10))
ReturnedItems.sort_values().plot(kind='barh', ax=ax1).set_title('Most Returned Items', fontsize=15)
ReturnCust.sort_values().plot(kind='barh', ax=ax2).set_title('Customers with Most Returns', fontsize=15)
ax1.set(xlabel='Quantity')
ax2.set(xlabel='Quantity')
plt.subplots_adjust(hspace=0.4)
plt.show()

2.3 The jointplot below displays a pairwise comparison between 'UnitPrice' and 'Quantity' for purchased products. This analysis illustrates an expected trend: as the price of a product increases, the quantity sold decreases. In contrast, customers tend to purchase products in larger quantities when they have lower prices. This relationship between price and quantity sold is a common and intuitive observation in sales analysis.

In [ ]:
#plotting the qunatity vs unitprice
Corr = sns.jointplot(x="Quantity", y="UnitPrice", data = Data_Cleaned[Data_Cleaned.FinalPrice>0], height = 7)
Corr.fig.suptitle("UnitPrice and Quantity Comparison", fontsize = 15, y = 1.1)
plt.show()

2.4 In the upcoming chart, we'll visualize the sales trend over the course of the year on a weekly basis. We'll achieve this by resampling our time series data to weeks and summing the values within each week. The chart will include two parts:

Weekly Sales: The first chart will show the weekly sales over the year.

Weekly Returns by Customers: The second chart will depict the weekly returns made by customers.
Here are some key observations:

After a noticeable dip in sales in

January, there's a consistent upward trend in sales throughout the year.
Regarding returns, they appear relatively stable throughout the year, with a minor increase, except for a spike in the second week of October.

In [ ]:
#resampling to get the weekly sales and returns
WeeklySale = Data_Cleaned[Data_Cleaned['Quantity']>0].Quantity.resample('W').sum()
WeeklyRet = Data_Cleaned[Data_Cleaned['Quantity']<0].Quantity.resample('W').sum().abs()
#creating the subplot
fig,[ax1, ax2] = plt.subplots(nrows=1,ncols=2, figsize = (15,5))
WeeklySale.plot(ax=ax1).set(xlabel="Month", ylabel="Quantity")
ax1.set_title("Weekly Sales Quantity", fontsize = 15)
WeeklyRet.plot(ax=ax2).set(xlabel="Month", ylabel="Quantity")
ax2.set_title("Weekly Returns Quantity", fontsize = 15)
plt.show()

2.5
In the upcoming chart, we'll explore the quantity of items sold and returned across different foreign countries. As the United Kingdom dominates sales and including it might obscure insights, we'll exclude it from the chart for a clearer and more informative visualization.

Here are the key observations:

It appears that our products were predominantly sold in the Netherlands.
On the other hand, the majority of returns were recorded in Ireland (EIRE). This information provides valuable insights into the distribution of sales and returns across foreign countries.

In [ ]:
#grouping data by the countries(except UK)
ByCountrySale = Data_Cleaned[(Data_Cleaned.Country != 'UNITED KINGDOM') & (Data_Cleaned.Quantity > 0)].groupby('Country')['Quantity'].sum()
ByCountryRet = Data_Cleaned[(Data_Cleaned.Country != 'UNITED KINGDOM') & (Data_Cleaned.Quantity < 0)].groupby('Country')['Quantity'].sum().abs()
#creating the subplot
fig, [ax1,ax2] = plt.subplots(nrows=2,ncols=1,figsize=(10,14))
ByCountrySale.plot(kind='bar', ax=ax1).set(ylabel = 'Quantity',xlabel='')
ax1.set_title('Sales', size=12, fontweight = 'bold')
ByCountryRet.plot(kind='bar', ax=ax2).set(ylabel = 'Quantity',xlabel='')
ax2.set_title('Returns', size=12, fontweight = 'bold')
plt.suptitle('Sales in Foreign Countries', fontsize = 15)
plt.subplots_adjust(hspace = 0.6)
plt.show()

2.6 Utilizing the day of the week in which items were sold, we can analyze the sales value by each day of the week. The data reveals the following pattern:

Thursday shows the highest sales value.
In contrast, Sunday records the lowest sales value.
This information highlights the variations in sales across different days of the week and can be valuable for optimizing sales and marketing strategies.

In [ ]:
#creating the pie chart
Data_Cleaned.groupby('Day of week')['FinalPrice'].sum().plot(kind = 'pie', autopct = '%.2f%%', figsize=(7,7)).set(ylabel='')
plt.title('Percantages of Sales Value by Day of Week', fontsize = 15)
plt.show()

2.7 We can identify our top customers based on the value they've contributed to the company. Additionally, we can show the countries from which these valuable customers originate. This analysis helps us recognize and appreciate our most significant customer relationships and understand their geographic distribution.

In [ ]:
# Filter the top 10 customers by total FinalPrice
Top10Customers = Data_Cleaned.groupby(['CustomerID', 'Country'])['FinalPrice'].sum().sort_values(ascending=False).head(10)

# Reset the index to make it a regular DataFrame
Top10Customers = Top10Customers.reset_index()

# Create a combined label for the y-axis
Top10Customers['CustomerInfo'] = Top10Customers['CustomerID'].astype(str) + ' (' + Top10Customers['Country'] + ')'

# Create the barplot with the combined y-axis label
plt.figure(figsize=(10, 6))
sns.barplot(x='FinalPrice', y='CustomerInfo', data=Top10Customers)
plt.xlabel('Total Value')
plt.title('Top 10 Customers by Sales Value and Country', fontsize=15)
plt.show()


2.8 In terms of future planning, it's valuable to know how many of our customers are repeat customers, which means they have made multiple purchases. In the plot below, we observe that approximately 70% of our customers fall into this category. This loyal customer base is important for business sustainability and growth.

Furthermore, in the second plot, we can identify which customers from different countries have the most repeat purchases. This information can help us tailor marketing strategies and customer retention efforts more effectively.

In [ ]:
# Group customers by the number of their visits and separate them
MostRepeat = Data_Cleaned.groupby(['CustomerID', 'Country'])['InvoiceNo'].nunique().sort_values(ascending=False)
repeat_customers = MostRepeat[MostRepeat != 1]
one_time_customers = MostRepeat[MostRepeat == 1]

# Calculate the proportions
repeat_customer_proportion = len(repeat_customers) / len(MostRepeat)
one_time_customer_proportion = len(one_time_customers) / len(MostRepeat)

# Create a subplot
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(15, 5), gridspec_kw={'width_ratios': [3, 1]})
plt.subplots_adjust(wspace=0.2)

# First subplot: Bar plot for top repeat customers
repeat_customers_df = repeat_customers.head(10).reset_index()
# Create a new column 'CustomerInfo' that combines 'CustomerID' and 'Country'
repeat_customers_df['CustomerInfo'] = repeat_customers_df['CustomerID'].astype(str) + ', ' + repeat_customers_df['Country']
sns.barplot(x='InvoiceNo', y='CustomerInfo', data=repeat_customers_df, ax=ax1)
ax1.set(xlabel='Number of Transactions (Repeats)', ylabel='Customer Info')

# Second subplot: Pie chart for customer distribution
proportions = [repeat_customer_proportion, one_time_customer_proportion]
ax2.pie(proportions, labels=['Repeat Customers', 'One-time Customers'], autopct='%.2f%%')
ax2.set(ylabel='')

# Overall title
plt.suptitle('Top Repeat Customers', fontsize=15)

# Show the plot
plt.show()


2.9 The plots below show the distributions of the 'Quantity' and 'UnitPrice' attributes, providing insights into the data patterns and characteristics of these features.

In [ ]:
#creating distribution plots
fig , [ax1,ax2] = plt.subplots(nrows=1,ncols=2,figsize=(12,4))
with sns.axes_style('dark'):
    sns.distplot(Data_Cleaned['Quantity'], ax=ax1)
    sns.distplot(Data_Cleaned['UnitPrice'], ax=ax2)
fig.suptitle('UnitPrice and Quantity Distribution', fontsize = 15)
plt.show()

2.10 In the last plot, we will use three features two show how the sales are distributed among different months and days of week. To show that, we will use seaborn's heatmap. The x-axis shows the day and the y-axis shows the month in which the items were bought. The color scale shows the total value of sales.

In [ ]:
HM_Data = Data_Cleaned.pivot_table(index = 'InvoiceMonth',columns = 'Day of week', values = 'FinalPrice', aggfunc='sum')
plt.figure(figsize = (10,6))
sns.heatmap(HM_Data, cmap = 'vlag').set(xlabel='', ylabel='')
plt.title('Sales Value per Month and Day of Week', fontsize = 15)
plt.show()

## 3. Association Rule Mining:
- 3.1 Identify frequently bought products together. Use the Apriori algorithm to extract meaningful association rules.
- 3.2 Based on the rules, suggest product bundling strategies to the retail store.

In [ ]:
#importing necessary libraries
from mlxtend.frequent_patterns import apriori, association_rules

#importing the dataset
Data_Cleaned = pd.read_csv('/content/drive/MyDrive/DAB303/Project2/OnlineRetail_Cleaned.csv', index_col = 'InvoiceDate')
Data_Cleaned.index = pd.to_datetime(Data_Cleaned.index, format = '%Y-%m-%d %H:%M')

#converting the data into the standard form
Baskets = Data_Cleaned.loc[(Data_Cleaned['Quantity']>0) ,['InvoiceNo','Description','Quantity']]
Baskets = Baskets.groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack(fill_value=0)
Baskets = (Baskets > 0)

#finding frequent itemsets and association rules
frequent_itemsets = apriori(Baskets, min_support=0.028, use_colnames=True)

rules = association_rules(frequent_itemsets, metric = 'confidence', min_threshold=0.3)
print(rules)

## 4. Customer Segmentation using Clustering:

In the final part of this demonstration, we will perform customer segmentation through clustering. Customer segmentation is a valuable process for categorizing our customers into distinct groups. This segmentation can be used for various purposes, such as informing future business strategies like seasonal discounts, special offers, and more. Additionally, it allows us to assign new customers to a cluster based on their behavior.

To get started, we'll import the required libraries and the dataset for this section. We'll begin by preparing the data for clustering. Our customers are divided into two main categories: those in the UK (which constitutes the majority) and those in foreign countries. We will separate them into UK and non-UK customers. Then, we'll proceed to group the data by individual customers and determine an appropriate method for aggregating the values associated with each customer. This aggregation and grouping are essential steps in preparing the data for the subsequent clustering analysis.

In [ ]:
!pip install kmodes

In [ ]:
from sklearn.preprocessing import StandardScaler
from kmodes.kprototypes import KPrototypes
from sklearn.metrics import silhouette_score
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as mpatches
%matplotlib inline


#importing the dataset
Data_Cleaned = pd.read_csv('/content/drive/MyDrive/DAB303/Project2/OnlineRetail_Cleaned.csv', index_col = 'InvoiceDate')
Data_Cleaned.index = pd.to_datetime(Data_Cleaned.index, format = '%Y-%m-%d %H:%M')

#grouping the data by customers and preparing for segmentation
Data = Data_Cleaned.copy()
Data['Country'] = Data['Country'].map(lambda x: 'UK' if x=='UNITED KINGDOM' else 'non-UK')
CustomerData = Data.groupby(['CustomerID','Country'], sort=False).agg({'Quantity':'mean','UnitPrice':'mean','InvoiceNo':'nunique','Description':'nunique'})
CustomerData.reset_index(inplace=True)
CustomerData.columns = ['CustomerID', 'UK?', 'Average Quantity', 'Average Price', 'Repeats', 'Product Variety']
CustomerData.head()

Before commencing the clustering process, it's wise to scale the numerical features. Scaling helps mitigate the potential negative effects of varying feature magnitudes and can also expedite the clustering process.

In our customer data, we have a categorical variable representing the customer's country. To accommodate this categorical data, we've chosen the k-prototypes algorithm, which can handle both numerical and categorical variables.

However, selecting the right number of clusters is crucial. We'll evaluate the clustering cost and the silhouette score to determine the optimal number of clusters. The silhouette score, in particular, provides insight into how similar data points are to their own cluster relative to other clusters, typically using Euclidean distance as a metric. This metric will guide us in selecting the appropriate number of clusters for our customer segmentation.

In [ ]:
#scaling the numerical features for clustering
Scaler = StandardScaler()
CustomerData.iloc[:,2:] = Scaler.fit_transform(CustomerData.iloc[:,2:])
syms = CustomerData.iloc[:,0].values.astype(str)
X = CustomerData.iloc[:,1:].values.astype(object)
#finding the optimal cluster_number k
for n in range(2,8):
    kproto = KPrototypes(n_clusters = n, init = 'Cao')
    clusters = kproto.fit_predict(X, categorical = [0])
    silhouette = silhouette_score(X[:,1:],clusters)
    print('number of clusters:', n)
    print('  cost: ',kproto.cost_)
    print('  average silhouette score: ',silhouette)

In [ ]:
# Data
number_of_clusters = [2, 3, 4, 5, 6, 7]
cost = [14261.69, 11688.49, 9374.43, 7136.42, 6353.18, 5633.88]
silhouette_score = [0.5060, 0.5232, 0.3604, 0.3686, 0.2594, 0.2906]

# Create a figure and axis
fig, ax1 = plt.subplots()

# Plot cost on the primary y-axis
ax1.set_xlabel('Number of Clusters')
ax1.set_ylabel('Cost', color='tab:blue')
ax1.plot(number_of_clusters, cost, color='tab:blue', marker='o', label='Cost')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.set_title('Clustering Metrics')

# Create a secondary y-axis for silhouette score
ax2 = ax1.twinx()
ax2.set_ylabel('Average Silhouette Score', color='tab:red')
ax2.plot(number_of_clusters, silhouette_score, color='tab:red', marker='s', label='Silhouette Score')
ax2.tick_params(axis='y', labelcolor='tab:red')

# Show legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Show the plot
plt.tight_layout()
plt.show()
